In [1]:
import keras.backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import Input, Add
from keras.layers.core import Activation, Lambda
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, UpSampling2D

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [3]:
def residual_block(x, nb_filter, block_idx, bn=True, weight_decay=0):

    # 1st conv
    name = "block%s_conv2D%s" % (block_idx, "a")
    W_reg = l2(weight_decay)
    r = Conv2D(nb_filter, (3, 3), padding="same", kernel_regularizer=W_reg, name=name)(x)
    if bn:
        r = BatchNormalization(axis=1, name="block%s_bn%s" % (block_idx, "a"))(r)
    r = Activation("relu", name="block%s_relu%s" % (block_idx, "a"))(r)

    # 2nd conv
    name = "block%s_conv2D%s" % (block_idx, "b")
    W_reg = l2(weight_decay)
    r = Conv2D(nb_filter, (3, 3), padding="same", kernel_regularizer=W_reg, name=name)(r)
    if bn:
        r = BatchNormalization(axis=1, name="block%s_bn%s" % (block_idx, "b"))(r)
    r = Activation("relu", name="block%s_relu%s" % (block_idx, "b"))(r)

    # Merge residual and identity
    x = Add(name="block%s_merge" % block_idx)([x, r])

    return x

In [4]:
def convolutional_block(x, block_idx, nb_filter, nb_conv, strides):

    # 1st conv
    for i in range(nb_conv):
        name = "block%s_conv2D_%s" % (block_idx, i)
        if i < nb_conv - 1:
            x = Conv2D(nb_filter, (3, 3), name=name, padding="same")(x)
            x = BatchNormalization(axis=1)(x)
            x = Activation("relu")(x)
        else:
            x = Conv2D(nb_filter, (3, 3), name=name, strides=strides, padding="same")(x)
            x = BatchNormalization(axis=1)(x)
            x = Activation("relu")(x)

    return x

In [12]:
def colorful(nb_classes, img_dim, batch_size, model_name="colorful"):

    nb_resblocks = 3
    block_idx = 0
    h, w = img_dim[:2]

    # First conv block
    x_input = Input(shape=img_dim, name="input")
    x = Conv2D(64, (3, 3), name="block%s_conv2d_0" % block_idx, padding="same")(x_input)
    x = Activation("relu", name="block%s_relu" % block_idx)(x)
    block_idx += 1

    # Residual blocks
    for idx, f in enumerate([64] * nb_resblocks):
        x = residual_block(x, f, block_idx, weight_decay=0)
        block_idx += 1

    # Final conv
    x = Conv2D(nb_classes, (1, 1), name="final_conv2D", padding="same")(x)

    # Reshape Softmax
    def output_shape(input_shape):
        return (batch_size, h, w, nb_classes + 1)

    def reshape_softmax(x):
        x = K.permute_dimensions(x, [0, 2, 3, 1])  # last dimension in number of filters
        x = K.reshape(x, (batch_size * h * w, nb_classes))
        x = K.softmax(x)
        # Add a zero column so that x has the same dimension as the target (313 classes + 1 weight)
        xc = K.zeros((batch_size * h * w, 1))
        x = K.concatenate([x, xc], axis=1)
        # Reshape back to (batch_size, h, w, nb_classes + 1) to satisfy keras' shape checks
        x = K.reshape(x, (batch_size, h, w, nb_classes + 1))
        return x

    ReshapeSoftmax = Lambda(lambda z: reshape_softmax(z), output_shape=output_shape, name="ReshapeSoftmax")
    x = ReshapeSoftmax(x)

    # Build model
    colorful = Model(inputs=[x_input], outputs=[x], name=model_name)

    return colorful


def load(nb_classes, img_dim, batch_size):

    model = colorful(nb_classes, img_dim, batch_size, model_name="colorful")

    return model

In [13]:
def load(nb_classes, img_dim, batch_size):

    model = colorful(nb_classes, img_dim, batch_size, model_name="colorful")

    return model

In [14]:
model = load(313, (256,256,1), 32)

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 256, 256, 1)   0                                            
____________________________________________________________________________________________________
block0_conv2d_0 (Conv2D)         (None, 256, 256, 64)  640         input[0][0]                      
____________________________________________________________________________________________________
block0_relu (Activation)         (None, 256, 256, 64)  0           block0_conv2d_0[0][0]            
____________________________________________________________________________________________________
block1_conv2Da (Conv2D)          (None, 256, 256, 64)  36928       block0_relu[0][0]                
___________________________________________________________________________________________

In [41]:
def reshape_softmax(x, batch_size=32, h=256, w=256, nb_classes=313):
        x = K.permute_dimensions(x, [0, 2, 3, 1])  # last dimension in number of filters
        x = K.reshape(x, (batch_size * h * w, nb_classes))
        x = K.softmax(x)
        # Add a zero column so that x has the same dimension as the target (313 classes + 1 weight)
        xc = K.zeros((batch_size * h * w, 1))
        x = K.concatenate([x, xc], axis=1)
        # Reshape back to (batch_size, h, w, nb_classes + 1) to satisfy keras' shape checks
        x = K.reshape(x, (batch_size, h, w, nb_classes + 1))
        return x

In [42]:
x_input = Input(shape=(256,256,1), name="input")
x = Conv2D(64, (3, 3), name="block%s_conv2d_0" % 1, padding="same")(x_input)
x = Activation("relu", name="block%s_relu" % 1)(x)
# Residual blocks
    for idx, f in enumerate([64] * nb_resblocks):
        x = residual_block(x, f, block_idx, weight_decay=0)
        block_idx += 1

In [43]:
test = Model(inputs=[x_input], outputs=[x], name='test')

In [44]:
test.summary()

Model: "test"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 256, 256, 1)]     0         
                                                                 
 block1_conv2d_0 (Conv2D)    (None, 256, 256, 64)      640       
                                                                 
 block1_relu (Activation)    (None, 256, 256, 64)      0         
                                                                 
Total params: 640
Trainable params: 640
Non-trainable params: 0
_________________________________________________________________


In [45]:
! keras --versuib

/bin/bash: keras: command not found


In [46]:
!pip --l


Usage:   
  pip3 <command> [options]

ambiguous option: --l (--local-log, --log, --log-file?)


In [48]:
!pip3 --list


Usage:   
  pip3 <command> [options]

no such option: --list


In [2]:
from keras import __version__
print('Using Keras version:', __version__, 'backend:', K.backend())

Using Keras version: 2.0.8 backend: tensorflow


In [2]:
!pip install q keras==2.0.8

In [6]:
!pip install q tensorflow==1.13.1

     |████████████████████████████████| 92.6 MB 74 kB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 367 kB 56.1 MB/s 
     |████████████████████████████████| 3.2 MB 43.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you 